In [15]:
import torch.optim as optim
from deepsplines.ds_modules import dsnn
import torch.nn as nn
import torch
import numpy as np
import wandb
from sklearn.model_selection import train_test_split

In [11]:
np.random.seed(1)
torch.manual_seed(1)
torch.set_default_dtype(torch.float)
device = torch.device('cpu' if torch.backends.mps.is_available() else 'mps')

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Deepsline California Housing",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.01,
    "architecture": "Deepline Linear",
    "dataset": "California Housing",
    "epochs": 1000,
    }
)

In [2]:
from sklearn.datasets import fetch_california_housing
 
data = fetch_california_housing()
print(data.feature_names)
 
X, y = data.data, data.target
# Do more data analysis 

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [3]:
opt_params = {
    'size': 51, #knots k+2
    'range_': 6,
    'init': 'leaky_relu',
    'save_memory': False
}


In [26]:
class DeepSplines(dsnn.DSModule):
    def __init__(self,opt_params,epochs,lr):

        super(DeepSplines,self).__init__()
        self.model = nn.Sequential(
                        nn.Linear(8, 24),
                        dsnn.DeepBSpline('fc', 24, **opt_params),
                        nn.Linear(24, 12),
                        dsnn.DeepBSpline('fc', 12, **opt_params),
                        nn.Linear(12, 8),
                        dsnn.DeepBSpline('fc', 8, **opt_params),
                        nn.Linear(8, 1)
                    )
        self.initialization(opt_params['init'], init_type='He')
        self.num_params = self.get_num_params()
        self.epochs = epochs
        self.lr = lr
    
    def forward(self,x):
        return self.model(x)
    
    def fit(self,X,y):
        self.train()
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters_deepspline(), lr=self.lr)
        for epoch in range(self.epochs):
            optimizer.zero_grad()
            y_pred = self.forward(X)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            wandb.log({"loss": loss.item()}, step=epoch)
            if epoch % 100 == 0:
                print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
        return loss.item()
    
    def predict(self,X,y):
        self.eval()
        criterion = nn.MSELoss()
        with torch.no_grad():
            y_pred = self.forward(X)
        loss= criterion(y_pred, y)
        #print('test loss: {}'.format(loss.item()))
        return loss.item()



In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

In [28]:
model=DeepSplines(opt_params,epochs=2000,lr=1e-4)
wandb.init(
    # set the wandb project where this run will be logged
    project="Deepsline California Housing",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.01,
    "architecture": "Deepline Linear",
    "dataset": "California Housing",
    "epochs": 1000,
    }
)
wandb.watch(model,log= "all",log_freq=10)
loss_tr=model.fit(X_train,y_train)

loss,█▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,1.19497


Epoch 0: train loss: 2048.640625
Epoch 100: train loss: 262.302978515625
Epoch 200: train loss: 24.150646209716797
Epoch 300: train loss: 3.652357339859009
Epoch 400: train loss: 2.0325727462768555
Epoch 500: train loss: 1.7896946668624878
Epoch 600: train loss: 1.6957175731658936
Epoch 700: train loss: 1.638900876045227
Epoch 800: train loss: 1.597341537475586
Epoch 900: train loss: 1.5654112100601196
Epoch 1000: train loss: 1.5399304628372192
Epoch 1100: train loss: 1.5191593170166016
Epoch 1200: train loss: 1.5016937255859375
Epoch 1300: train loss: 1.4867788553237915
Epoch 1400: train loss: 1.4739432334899902
Epoch 1500: train loss: 1.4625310897827148
Epoch 1600: train loss: 1.4523838758468628
Epoch 1700: train loss: 1.4435290098190308
Epoch 1800: train loss: 1.4354238510131836
Epoch 1900: train loss: 1.4280016422271729


wandb: Network error (ConnectTimeout), entering retry loop.
